# Phase-2 Handling Missing Values

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import warnings;
warnings.filterwarnings("ignore")
import re
#pd.set_option("display.max_columns",None)
#pd.set_option("display.max_rows",None)

c:\Users\Necip\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df=pd.read_csv("Capstone_new.csv")

### 1) Body Type

In [6]:
df["body_type"].value_counts(dropna=False)

body_type
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64

In [7]:
df[df["body_type"] == np.nan]

,make_model,body_type,price,vat,km,registration,prev_owner,hp,Type,Next Inspection,...,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,Gears,Country version,upholstery_material,upholstery_color,Consumption_combined,Consumption_city,Consumption_country


In [9]:
df.groupby('make_model')['body_type'].transform(lambda x: x.mode()[0])

0        Sedans
1        Sedans
2        Sedans
3        Sedans
4        Sedans
          ...  
15914       Van
15915       Van
15916       Van
15917       Van
15918       Van
Name: body_type, Length: 15919, dtype: object

In [10]:
df["body_type"].fillna(df.groupby('make_model')['body_type'].transform(lambda x: x.mode()[0]), inplace=True)

### 2) Registration

In [12]:
df.loc[((df['Type'] == 'New') & df.registration.isnull()), 'registration'] 

122     NaN
710     NaN
734     NaN
741     NaN
743     NaN
         ..
15892   NaN
15902   NaN
15907   NaN
15912   NaN
15914   NaN
Name: registration, Length: 1547, dtype: float64

In [13]:
df.loc[((df['Type'] == 'New') & df.registration.isnull()), 'registration'] = 2019 

In [14]:
df.groupby('registration').km.mean()

registration
2016.0    77442.520958
2017.0    41754.940709
2018.0    18035.239072
2019.0     1653.107634
Name: km, dtype: float64

* Since the registration of all new cars is 2019, null values of year column of new cars filled with 2019.

In [15]:
df.loc[(df['registration'].isnull()) & (df.km <= 5000),'registration'] =2019

In [16]:
df.loc[(df['registration'].isnull()) & (df.km > 50000),'registration'] =2016

In [17]:
df.loc[(df['registration'].isnull()) & (df.km < 19000),'registration'] =2018

In [18]:
df.loc[(df['registration'].isnull()) & (df.km < 40000),'registration'] =2017

* Rest of the null values were filled according to km of autos.

In [19]:
df[(df['make_model'] == 'Audi A3') & (df['body_type'] == 'Sedans')].groupby('registration').price.mean()

registration
2016.0    16702.052388
2017.0    19970.030631
2018.0    22544.715092
2019.0    24859.973913
Name: price, dtype: float64

In [20]:
df[(df['make_model'] == 'Opel Insignia')].groupby('registration').price.mean()

registration
2016.0    13606.709507
2017.0    16995.140917
2018.0    21390.167750
2019.0    32103.197015
Name: price, dtype: float64

In [21]:
df.loc[(df['registration'].isnull())]

,make_model,body_type,price,vat,km,registration,prev_owner,hp,Type,Next Inspection,...,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,Gears,Country version,upholstery_material,upholstery_color,Consumption_combined,Consumption_city,Consumption_country
5237,Audi A3,Sedans,25400,NaN,NaN,NaN,NaN,85.0,NaN,NaN,...,"['Bluetooth', 'Hands-free equipment']",['Alloy wheels'],"['ABS', 'Central door lock', 'Driver-side airb...",7.0,NaN,Cloth,NaN,3.9,4.1,3.7
5329,Audi A3,Sedans,24900,NaN,NaN,NaN,NaN,85.0,Pre-registered,NaN,...,"['Bluetooth', 'Hands-free equipment']",['Alloy wheels'],"['ABS', 'Central door lock', 'Driver-side airb...",7.0,NaN,Cloth,NaN,3.9,4.1,3.7
12550,Opel Insignia,Sedans,33800,NaN,NaN,NaN,NaN,100.0,Employee's car,NaN,...,"['Bluetooth', 'Hands-free equipment', 'MP3', '...","['Alloy wheels', 'Sport package', 'Touch scree...","['ABS', 'Blind spot monitor', 'Central door lo...",6.0,NaN,Part leather,Black,5.4,7.0,4.6
12882,Opel Insignia,Station wagon,31318,VAT deductible,NaN,NaN,NaN,100.0,Used,NaN,...,"['Bluetooth', 'On-board computer']","['Alloy wheels', 'Sport seats']","['ABS', 'Central door lock', 'Driver-side airb...",6.0,NaN,Full leather,Black,4.3,NaN,NaN


In [22]:
df['registration']=df['registration'].fillna(2019)

In [23]:
df['registration'] = 2019 - df['registration']

df = df.rename(columns = {'registration': 'age'})

* year column converted to age column

### 3) VAT

In [15]:
df.vat.value_counts(dropna=False)

VAT deductible      10980
NaN                  4513
Price negotiable      426
Name: vat, dtype: int64

In [16]:
df['vat'] = df['vat'].fillna('VAT undeductible') 

* All null values of vat column assigned as 'VAT undeductible'

### 4) KM

In [24]:
median_group_km = df.groupby(['make_model','body_type','age']).km.transform(lambda x: x.median())

In [25]:
df["km"].fillna(median_group_km, inplace=True)

In [26]:
df["km"].fillna(df.groupby(['make_model','age']).km.transform(lambda x: x.median()), inplace=True)

* km column filled with groups age, make_model and body_type

### 5) Previous Owner

In [29]:
index=df[df["km"]<10].index

In [30]:
df.loc[index,"prev_owner"]=0.0

In [31]:
df.loc[(df["Type"]=="New") & (df["prev_owner"].isnull()),"prev_owner"]=1.0

In [32]:
df.loc[((df["age"]==2019) | (df["km"]<5000)) & (df["prev_owner"].isnull()),"prev_owner"]=1.0

In [33]:
mode_group_po = df.groupby(['age'])['prev_owner'].transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [34]:
df["prev_owner"].fillna(mode_group_po, inplace=True)

### hp 

In [35]:
mode_hp = df[~(df['Displacement'].isnull())]\
              .groupby(['make_model','body_type','Displacement'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [36]:
df.loc[~(df['Displacement'].isnull()),"hp"] = df.loc[~(df['Displacement'].isnull()),"hp"].fillna(mode_hp)

In [37]:
mode_hp1 = df[~(df['Cylinders'].isnull())]\
              .groupby(['make_model','body_type','Cylinders'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [38]:
df.loc[~(df['Cylinders'].isnull()),"hp"] = df.loc[~(df['Cylinders'].isnull()),"hp"].fillna(mode_hp1)

In [39]:
mode_hp2 = df.groupby(['make_model','body_type'])['hp']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [40]:
df['hp'] = df['hp'].fillna(mode_hp2)

 ### 7) Type

In [41]:
df['Type'] = df['Type'].fillna('Used')

### 8) Next Inspection

In [33]:
df.drop(["Next Inspection"],axis=1,inplace=True)

### 9) Inspection New

In [34]:
df['Inspection new'].value_counts(dropna=False)

NaN    11987
Yes     3932
Name: Inspection new, dtype: int64

In [35]:
df['Inspection new'] = df['Inspection new'].fillna('No')

### 10) Warranty(months)

In [36]:
df['Warranty(months)'] = df['Warranty(months)'].fillna(0.0)

* warranty column was filled with 0's

### 11) Body Color

In [37]:
mode_color = df.groupby(['make_model','body_type'])['Body Color']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [38]:
df['Body Color'] = df['Body Color'].fillna(mode_color)

In [39]:
mode_color1 = df.groupby(['make_model'])['Body Color']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [40]:
df['Body Color'] = df['Body Color'].fillna(mode_color1)

* body_color column was filled according to groups make_model, body_type

### 11) Paint Type

In [41]:
mode_paint = df.groupby(['make_model','body_type'])['Paint Type']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [42]:
df['Paint Type'] = df['Paint Type'].fillna(mode_paint)

In [43]:
mode_paint1 = df.groupby(['make_model'])['Paint Type']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [44]:
df['Paint Type'] = df['Paint Type'].fillna(mode_paint1)

* paint_type column was filled according to groups make_model, body_type

### 12) Body Color Original

In [45]:
df.drop(["Body Color Original"],axis=1,inplace=True)

### 13) Nr. of Doors

In [46]:
mode_nr_doors = df.groupby(['make_model','body_type'])['Nr. of Doors']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [47]:
df['Nr. of Doors'] = df['Nr. of Doors'].fillna(mode_nr_doors)

In [48]:
mode_nr_doors1 = df.groupby(['make_model'])['Nr. of Doors']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [49]:
df['Nr. of Doors'] = df['Nr. of Doors'].fillna(mode_nr_doors1)

* nr. of seats column was filled according to groups make_model, body_type

### 14) Nr. of Seats

In [50]:
mode_nr_seats = df.groupby(['make_model','body_type'])['Nr. of Seats']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [51]:
df['Nr. of Seats'] = df['Nr. of Seats'].fillna(mode_nr_seats)

In [52]:
mode_nr_seats1 = df.groupby(['make_model'])['Nr. of Seats']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [53]:
df['Nr. of Seats'] = df['Nr. of Seats'].fillna(mode_nr_seats1)

* nr_of_seats column was filled according to groups make_model, body_type

### Model Code

In [54]:
df.drop(['Model Code'],axis=1,inplace=True)

### 15) Gearing Type

In [55]:
df['Gearing Type'].value_counts(dropna=False)

Manual            8153
Automatic         7297
Semi-automatic     469
Name: Gearing Type, dtype: int64

### 16) Displacement

In [56]:
mode_displacement = df.groupby(['make_model','body_type','hp'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [57]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement)

In [58]:
mode_displacement1 = df.groupby(['make_model','body_type'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [59]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement1)

In [60]:
mode_displacement2 = df.groupby(['make_model'])['Displacement']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [61]:
df['Displacement'] = df['Displacement'].fillna(mode_displacement2)

* Displacement column was filled according to groups hp, make_model, body_type

### 16) Weight

In [62]:
mode_weight = df.groupby(['make_model', 'Displacement'])['Weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [63]:
df['Weight'] = df['Weight'].fillna(mode_weight)

In [64]:
mode_weight1 = df.groupby(['make_model'])['Weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [65]:
df['Weight'] = df['Weight'].fillna(mode_weight1)

In [66]:
mode_weight2 = df.groupby(['make_model', 'hp'])['Weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [67]:
df['Weight'] = df['Weight'].fillna(mode_weight2)

In [68]:
mode_weight3 = df.groupby(['body_type'])['Weight']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [69]:
df['Weight'] = df['Weight'].fillna(mode_weight3)

* weight column was filled according to groups hp, make_model, body_type

### 17) Cylinders

In [70]:
mode_cylinders = df.groupby(['make_model', 'Displacement'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [71]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders)

In [72]:
mode_cylinders1 = df.groupby(['make_model', 'Weight'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [73]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders1)

In [74]:
mode_cylinders2 = df.groupby(['make_model'])['Cylinders']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [75]:
df['Cylinders'] = df['Cylinders'].fillna(mode_cylinders2)

In [76]:
df['Cylinders'] = df['Cylinders'].fillna(4.0)

* cylinders column was filled according to groups displacement, weight, make_mode

### 18) Drive Chain

In [77]:
mode_drive_chain = df.groupby(['make_model', 'body_type'])['Drive chain']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [78]:
df['Drive chain'] = df['Drive chain'].fillna(mode_drive_chain)

In [79]:
mode_drive_chain1 = df.groupby(['make_model'])['Drive chain']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [80]:
df['Drive chain'] = df['Drive chain'].fillna(mode_drive_chain1)

* drive_chain column was filled according to groups make_model and body type

### 19) Fuel

In [81]:
df['Fuel'].value_counts(dropna=False)

Benzine    8549
Diesel     7299
Gas          64
Others        7
Name: Fuel, dtype: int64

### 20) CO2 Emission(g CO2/km)

In [82]:
mode_CO2 = df.groupby(['make_model','Displacement'])['CO2 Emission(g CO2/km)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [83]:
df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2)

In [84]:
mode_CO2_1 = df.groupby(['make_model'])['CO2 Emission(g CO2/km)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [85]:
df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2_1)

In [86]:
mode_CO2_2 = df.groupby(['body_type'])['CO2 Emission(g CO2/km)']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [87]:
df['CO2 Emission(g CO2/km)'] = df['CO2 Emission(g CO2/km)'].fillna(mode_CO2_2)

* 'CO2 Emission(g CO2/km)' column was filled according to groups make_model,displacement and body type

### 21) Emission Class

In [88]:
mode_Emision_C= df.groupby(['CO2 Emission(g CO2/km)'])['Emission Class']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [89]:
df['Emission Class'] = df['Emission Class'].fillna(mode_Emision_C)

In [90]:
mode_Emision_C1= df.groupby(['age'])['Emission Class']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [91]:
df['Emission Class'] = df['Emission Class'].fillna(mode_Emision_C1)

* emission class column was filled according to groups co2 emission and age

### 22) Gears

In [92]:
mode_Gears= df.groupby(['make_model','Gearing Type'])['Gears']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [93]:
df['Gears'] = df['Gears'].fillna(mode_Gears)

In [94]:
mode_Gears1= df.groupby(['Gearing Type'])['Gears']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [95]:
df['Gears'] = df['Gears'].fillna(mode_Gears1)

* gears column was filled according to groups make model and gear type

### 23) Country version

In [96]:
df.drop(['Country version'], axis=1, inplace=True)

### 24) Upholstery Material

In [97]:
df['upholstery_material'].value_counts(dropna=False)

Cloth           8423
NaN             4503
Part leather    1499
Full leather    1009
Other            368
Velour            60
alcantara         57
Name: upholstery_material, dtype: int64

### 25) Upholstrey Color

In [98]:
df.drop(['upholstery_color'], axis=1, inplace=True)

### 26) Consumption Combined

In [99]:
coms = ((df['Consumption_city']+df['Consumption_country'])/2)

In [100]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(coms)

In [101]:
mode_Cons= df.groupby(['CO2 Emission(g CO2/km)'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [102]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons)

In [103]:
mode_Cons1= df.groupby(['Displacement'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [104]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons1)

In [105]:
mode_Cons2= df.groupby(['make_model'])['Consumption_combined']\
              .transform(lambda x: x.mode()[0] if list(x.mode()) != [] else np.nan)

In [106]:
df['Consumption_combined'] = df['Consumption_combined'].fillna(mode_Cons2)

* comsumption column first was filled with the mean of 'consumption_city','consumption_country' columns. Then was filled according to the column make_model.

## Filled data was saved to a csv file called "missings_filled_step2.csv"

In [107]:
df.to_csv("missings_filled_step2.csv", index=False)